In [1]:
from azure.ai.ml import MLClient
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

In [2]:
keyVaultName = "mobilepricinga6945442583"
KVUri = f"https://{keyVaultName}.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

In [3]:
subs_id = client.get_secret("subscription-id").value
rg_name = client.get_secret("ml-resource-group").value
ws_name = client.get_secret("ml-workspace-name").value

In [4]:
ml_client = MLClient(
    credential=credential,
    subscription_id=subs_id,
    resource_group_name=rg_name,
    workspace_name=ws_name,
)

In [6]:
ml_client.models.download(name="mobile_pricing_model",version="2",download_path="./")

## Create a local endpoint

In [7]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

In [8]:
import uuid
# endpoint_name = "mobile-pricing-endpoint-" + str(uuid.uuid4())[:8]
endpoint_name = "mobile-pricing-endpoint-a116c8ce"

In [9]:
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="Endpoint for mobile pricing model",
    auth_mode="key"
)

In [20]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Updating local endpoint (mobile-pricing-endpoint-a116c8ce) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Failed', 'scoring_uri': None, 'openapi_uri': None, 'name': 'mobile-pricing-endpoint-a116c8ce', 'description': 'Endpoint for mobile pricing model', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/consulting/setu/mlops_bitbucket/content/01_Mlops_Using_Cloud_Tools/deployment_azure_ml/notebooks'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x121978970>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

## Create a local deployment

In [12]:
from azure.ai.ml.entities import ManagedOnlineDeployment,Model,Environment,CodeConfiguration

In [13]:
model = Model(path="./mobile_pricing_model/mobile_pricing_model/model.pkl")
env = Environment(
    conda_file="./mobile_pricing_model/mobile_pricing_model/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

In [15]:
ml_client.online_deployments.begin_create_or_update(blue_deployment,local=True)

Updating local deployment (mobile-pricing-endpoint-a116c8ce / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest
 ---> 712ae957f2d3
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 1f313b4a21cc
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> f09d8c13bac9
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> e88279d0c75c
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> 962f0f21c27d
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> 73341bbb01af
Successfully built 73341bbb01af
Successfully tagged mobile-pricing-endpoint-a116c8ce:blue

Starting up endpoint...Done (0m 20s)


ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'mobile-pricing-endpoint-a116c8ce', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/consulting/setu/mlops_bitbucket/content/01_Mlops_Using_Cloud_Tools/deployment_azure_ml/notebooks'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x1219c6440>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '1e570cc5f3ed6d7077651f8dbd550c0e', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/

In [17]:
endpoint = ml_client.online_endpoints.get(endpoint_name, local=True)
scoring_uri = endpoint.scoring_uri

In [19]:
endpoint

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Failed', 'scoring_uri': None, 'openapi_uri': None, 'name': 'mobile-pricing-endpoint-a116c8ce', 'description': 'Endpoint for mobile pricing model', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/consulting/setu/mlops_bitbucket/content/01_Mlops_Using_Cloud_Tools/deployment_azure_ml/notebooks'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x12190a8c0>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})